In [0]:
%%time
from google.colab import drive
drive.mount('/content/drive')
%cd "drive/My Drive/pytorch/評価"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/pytorch/評価
CPU times: user 333 ms, sys: 105 ms, total: 438 ms
Wall time: 19.6 s


In [0]:
%%time
! npm install -g localtunnel
!python3 -m pip install visdom
!python3 -m visdom.server -port 8076 >> visdomlog.txt 2>&1 &
!lt --port 8076 >> url.txt 2>&1 &

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.0
added 35 packages from 21 contributors in 1.959s
     |████████████████████████████████| 686kB 3.4MB/s 
     |████████████████████████████████| 204kB 11.5MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=734563c63f66a1c231cae96b44e7b8e0e95b3050fed3cbd64df8488579ded28e
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=2f8713503d05a4e38636a831f79aa9cf8c1ffa1fb74f35aa0b202cdf97257406
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
CPU times: user 112 ms, sys: 29.3 ms, total: 141 ms
Wall time: 17.3 s


In [0]:
%%time
import time
time.sleep(5)
! cat url.txt

CPU times: user 10.5 ms, sys: 4.48 ms, total: 15 ms
Wall time: 5.97 s


In [0]:
from lib import network_finetuning as network
from lib import architecture
from lib import dataloader as dl
from lib import optimizer
from lib import trainer

import torchvision

In [0]:
%%time
net = network.ResNet50(class_size=10)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [0]:
%%time
batch_size = 128
transform = dl.pattern_transform(resize=224, HorizontalFlip=True, VerticalFlip=True, Rotation=True, Perspective=True, Crop=True, Erasing=True)

train_dataset = torchvision.datasets.ImageFolder(root='./data/sign_mnist/train', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root='./data/sign_mnist/test', transform=dl.val_transform())
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size)

In [0]:
%%time
batch_size = 128
transform = dl.pattern_transform(resize=224, HorizontalFlip=True, VerticalFlip=True, Rotation=True, Perspective=True, Crop=True, Erasing=True)
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=dl.val_transform())
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size)

[Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR), RandomHorizontalFlip(p=0.5), RandomVerticalFlip(p=0.5), RandomRotation(degrees=(-180, 180), resample=False, expand=False), RandomPerspective(p=0.5), RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR), ToTensor(), <torchvision.transforms.transforms.RandomErasing object at 0x7f6a9e2c1f60>, Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))]
Files already downloaded and verified
Files already downloaded and verified
CPU times: user 1.57 s, sys: 608 ms, total: 2.18 s
Wall time: 8.02 s


In [0]:
%%time
model = architecture.CNN_Architecture(net)

CPU times: user 2.94 ms, sys: 34.2 ms, total: 37.2 ms
Wall time: 63.5 ms


In [0]:
%%time
model.train(data_loader, epoch_count=50)

Setting up a new session...


cuda
----------
Epoch 1/50 2020-01-26 03:42:45.097871
val Loss: 1.6538 Acc: 0.4276
----------
Epoch 2/50 2020-01-26 03:43:15.207714
train Loss: 1.6208 Acc: 0.4196
val Loss: 1.7886 Acc: 0.4273
----------
Epoch 3/50 2020-01-26 03:52:09.081864
train Loss: 1.5801 Acc: 0.4330
val Loss: 1.6163 Acc: 0.4590
----------
Epoch 4/50 2020-01-26 04:01:02.878773
train Loss: 1.5458 Acc: 0.4497
val Loss: 1.6164 Acc: 0.4670
----------
Epoch 5/50 2020-01-26 04:09:56.680351
train Loss: 1.4985 Acc: 0.4639
val Loss: 1.6672 Acc: 0.4665
----------
Epoch 6/50 2020-01-26 04:18:50.235827
train Loss: 1.4669 Acc: 0.4773
val Loss: 1.5523 Acc: 0.4928
----------
Epoch 7/50 2020-01-26 04:27:44.112556
train Loss: 1.4300 Acc: 0.4887
val Loss: 1.6166 Acc: 0.4945
----------
Epoch 8/50 2020-01-26 04:36:37.883889
train Loss: 1.3928 Acc: 0.5037
val Loss: 1.3931 Acc: 0.5321
----------
Epoch 9/50 2020-01-26 04:45:31.053912
train Loss: 1.3571 Acc: 0.5169
val Loss: 1.3625 Acc: 0.5448
----------
Epoch 10/50 2020-01-26 04:54:24.59

In [0]:
%%time
model.predict(data_loader['val'])

検証画像に対しての正解率： 76 %
CPU times: user 18.5 s, sys: 11.8 s, total: 30.3 s
Wall time: 31.5 s
